In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from cpyment import CModel
from SEIR_x_UD import seirxud_abm_gill, SEIRxUD

In [3]:
N = 1000
I0 = 0.01
c = 13
beta = 0.025
alpha = 0.2
gamma = 0.0714
theta = 0.1
kappa = 0.03
eta = 0.75
chi = 0.25

R0 = beta*c/gamma
print('R0 = {0}'.format(R0))

R0 = 4.551820728291316


In [4]:
tmax = 300

params = {
    'c': c,
    'beta': beta,
    'alpha': alpha,
    'gamma': gamma,
    'theta': theta,
    'kappa': kappa,
    'eta': eta,
    'chi': chi
}

model = SEIRxUD(N=N, tmax=tmax, I0=I0, **params)

trajsGill = model.run_abm()

trajsGillAvg = np.average(trajsGill, axis=0)
trajsGillStd = np.std(trajsGill, axis=0)

1/10


TypeError: too many arguments: expected 11, got 12

In [ ]:
t0i = 0
delay = model.t[t0i]

traj = model.run_cmodel(t0=delay, y0=trajsGillAvg[:,t0i], etadamp=1)
trajNoCT = model.run_cmodel(etadamp=0)

In [ ]:
fig, ax = plt.subplots()
t = model.t.copy()

colors = {
    'S': np.array((0.0, 0.5, 1.0)),
    'E': np.array((1.0, 0.6, 0.0)),
    'I': np.array((1.0, 0.2, 0.1)),
    'R': np.array((0.0, 0.7, 0.4)),
}



#ax.set_xlim(0,t[10])
#ax.set_ylim(-1, 15)
stdn = 1

for i, s in enumerate('SEIR'):
    for j, d in enumerate('UD'):
        col = colors[s]*(1 if j == 0 else 0.5)
        ax.plot(t[t0i:], traj['y'][:,2*i+j], c=col, label=s+d, lw=2.0)
        ax.plot(t, trajNoCT['y'][:,2*i+j], c=col, lw=2.0, ls='--')
        ax.plot(t, trajsGillAvg[2*i+j], c=col, lw=0.7)
        ax.fill_between(t, 
                        trajsGillAvg[2*i+j]+stdn*trajsGillStd[2*i+j], 
                        trajsGillAvg[2*i+j]-stdn*trajsGillStd[2*i+j], 
                        color=list(col) + [0.3])
    
ax.axhline((1-1/R0)*N, c=(0,0,0), lw=0.5, ls='--')
ax.legend()

In [ ]:
# Total fraction of diagnosed people

fig, ax = plt.subplots()

ax.set_title('Percentage of diagnosed people')

ax.set_xlabel('t')
ax.set_ylabel('*D (%)')

ax.plot(t[t0i:], np.sum(traj['y'][:,1::2], axis=1)*100/N, label='ODE')
ax.plot(t, np.sum(trajsGillAvg[1::2], axis=0)*100/N, label='ABM')

ax.legend()

In [ ]:
from scipy.integrate import odeint

In [ ]:
# Nonlinear version
def ode_nonlin(y, t, N=1000,
                     beta=0.05,
                     alpha=0.2,
                     gamma=0.1,
                     theta=0.0,
                     kappa=0.05,
                     eta=0,
                     chi=0):
    dydt = y*0
    SU, SD, EU, ED, IU, ID, RU, RD = y
    
    pCI = 1-(1-theta*IU/N)**(c/gamma)
    dSUEU = beta*c/N*SU*IU
    dSUSD = chi*eta*(1-beta)*pCI*SU
    dSDSU = kappa*SD
    dEUIU = alpha*EU
    dEUED = eta*chi*theta*EU
    dEDID = alpha*ED
    dIUID = theta*(1+eta*chi)*IU
    dIURU = gamma*IU
    dIDRD = gamma*ID
    dRURD = chi*eta*pCI*RU
    dRDRU = kappa*RD
    
    dydt[0] = -dSUEU-dSUSD + dSDSU
    dydt[1] = dSUSD-dSDSU
    dydt[2] = dSUEU-dEUIU-dEUED
    dydt[3] = dEUED-dEDID
    dydt[4] = dEUIU-dIUID-dIURU
    dydt[5] = dEDID+dIUID-dIDRD
    dydt[6] = dIURU-dRURD+dRDRU
    dydt[7] = dIDRD-dRDRU+dRURD
    
    return dydt

In [ ]:
y0 = np.zeros(8)
y0[0] = N*(1-I0)
y0[4] = N*I0
sol = odeint(ode_nonlin, y0, model.t, args=(N, beta, alpha, gamma, theta, kappa, eta, chi))

In [ ]:
fig, ax = plt.subplots()

ax.plot(model.t, sol)
ax.plot(model.t, traj['y'])